# **MNIST AutoEncoder notebook**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Deyht/AI_astro_ED_AAIF/blob/main/codes/CNN/autoencoder/mnist_autoencoder_CIANNA.ipynb)

---


**Link to the CIANNA github repository**
https://github.com/Deyht/CIANNA

### **CIANNA installation**

#### Query GPU allocation and properties

If nvidia-smi fail, it might indicate that you launched the colab session whithout GPU reservation.  
To change the type of reservation go to "Runtime"->"Change runtime type" and select "GPU" as your hardware accelerator.

In [ ]:
%%shell

nvidia-smi

cd /content/

git clone https://github.com/NVIDIA/cuda-samples/

cd /content/cuda-samples/Samples/1_Utilities/deviceQuery/

cmake CMakeLists.txt

make SMS="50 60 70 80"

./deviceQuery | grep Capability | cut -c50- > ~/cuda_infos.txt
./deviceQuery | grep "CUDA Driver Version / Runtime Version" | cut -c57- >> ~/cuda_infos.txt

cd ~/

If you are granted a GPU that supports high FP16 compute scaling (e.g the Tesla T4), it is advised to change the mixed_precision parameter in the prediction to "FP16C_FP32A".  
See the detail description on mixed precision support with CIANNA on the [Systeme Requirements](https://github.com/Deyht/CIANNA/wiki/1\)-System-Requirements) wiki page.

#### Clone CIANNA git repository

In [ ]:
%%shell

cd /content/

git clone https://github.com/Deyht/CIANNA

cd CIANNA

#### Compiling CIANNA for the allocated GPU generation

There is no guaranteed forward or backward compatibility between Nvidia GPU generation, and some capabilities are generation specific. For these reasons, CIANNA must be provided the platform GPU generation at compile time.
The following cell will automatically update all the necessary files based on the detected GPU, and compile CIANNA.

In [ ]:
%%shell

cd /content/CIANNA

mult="10"
cat ~/cuda_infos.txt
comp_cap="$(sed '1!d' ~/cuda_infos.txt)"
cuda_vers="$(sed '2!d' ~/cuda_infos.txt)"

lim="11.1"
old_arg=$(awk '{if ($1 < $2) print "-D CUDA_OLD";}' <<<"${cuda_vers} ${lim}")

sm_val=$(awk '{print $1*$2}' <<<"${mult} ${comp_cap}")

gen_val=$(awk '{if ($1 >= 80) print "-D GEN_AMPERE"; else if($1 >= 70) print "-D GEN_VOLTA";}' <<<"${sm_val}")

sed -i "s/.*arch=sm.*/\\t\tcuda_arg=\"\$cuda_arg -D CUDA -D comp_CUDA -lcublas -lcudart -arch=sm_$sm_val $old_arg $gen_val\"/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" src/python_module_setup.py

./compile.cp CUDA PY_INTERF

mv src/build/lib.linux-x86_64-* src/build/lib.linux-x86_64

#### Testing CIANNA installation

**IMPORTANT NOTE**   
CIANNA is mainly used in a script fashion and was not designed to run in notebooks. Every cell code that directly invokes CIANNA functions must be run as a script to avoid possible errors.  
To do so, the cell must have the following structure.

```
%%shell

cd /content/CIANNA

python3 - <<EOF

[... your python code ...]

EOF
```

This syntax allows one to easily edit python code in the notebook while running the cell as a script. Note that all the notebook variables can not be accessed by the cell in this context.


### AutoEncoder training

In [ ]:
%%shell

cd /content/

python3 - <<EOF

import numpy as np
import matplotlib.pyplot as plt
from threading import Thread
import os, sys, glob

import albumentations as A
import cv2

sys.path.insert(0,glob.glob('/content/CIANNA/src/build/lib.*/')[-1])
import CIANNA as cnn

def i_ar(int_list):
	return np.array(int_list, dtype="int")

def f_ar(float_list):
	return np.array(float_list, dtype="float32")

if(not os.path.isdir("mnist_dat")):
	os.system("wget https://share.obspm.fr/s/EkYR5B2Wc2gNis3/download/mnist.tar.gz")
	os.system("tar -xvzf mnist.tar.gz")

if(not os.path.isdir("fig")):
	os.system("mkdir fig")

#Add data augmentation to increase the diversity
transform = A.Compose([
	A.Affine(scale={"x":[0.90,1.0],"y":[0.90,1.0]}, rotate=[-10,10], fit_output=True, mode=cv2.BORDER_CONSTANT, cval=0.025),
	A.LongestMaxSize(max_size=28),
	A.PadIfNeeded(min_height=28, min_width=28, value=0.025),
])

image_size = 28

data = np.fromfile("mnist_dat/mnist_input.dat", dtype="float32")
data = np.reshape(data, (80000,image_size*image_size))*0.95 + 0.025

#Train on the combination of tain and valid dataset from MNIST to improve diversity
nb_train = 70000

data_train = data[:nb_train,:]
data_valid = data[nb_train:,:]

iter_size = 65536

def create_batch(visual=0):

	data = np.zeros((iter_size,image_size*image_size), dtype="float32")

	if(visual):
		fig, axs = plt.subplots(4, 5, figsize=(10,8), dpi=300, constrained_layout=True)

	for i in range(0,iter_size):

		i_d = int(np.random.random()*nb_train)

		if(np.random.random() < 0.5):
			transformed = transform(image=np.reshape(data_train[i_d],(image_size,image_size)))
			patch = transformed['image']
		else:
			patch = np.reshape(data_train[i_d],(image_size,image_size))

		data[i,:] = patch.flatten("C")

		if(visual and i < 4*5):
			axs[i//5][i%5].imshow(patch, vmax=1.0, vmin=0.0, interpolation="bilinear", cmap="Greys")
			axs[i//5][i%5].axis('off')

	if(visual):
		plt.savefig("train_img.png", dpi=300)

	return data

def data_augm_fct():
	data_augm = create_batch(0)
	cnn.delete_dataset("TRAIN_buf", silent=1)
	cnn.create_dataset("TRAIN_buf", iter_size, data_augm, data_augm, silent=1)
	return


cnn.init(in_dim=i_ar([image_size,image_size]), in_nb_ch=1, out_dim=image_size*image_size,
		bias=0.1, b_size=32, comp_meth="C_CUDA", #Change to C_BLAS or C_NAIV
		dynamic_load=1, mixed_precision="FP16C_FP32A")

data_augm = create_batch(1)
cnn.create_dataset("TRAIN", size=iter_size, input=data_augm, target=data_augm)
cnn.create_dataset("VALID", size=80000-nb_train, input=data_valid, target=data_valid)
cnn.create_dataset("TEST" , size=32, input=data_valid[:32], target=data_valid[:32])

a_relu = cnn.relu(leaking=0.1, saturation=640000.0)
a_lin = cnn.relu(leaking=1.0, saturation=640000.0)

load_step = 0
if(load_step > 0):
	cnn.load("net_save/net0_s%04d.dat"%(load_step), load_step)
else:

  #Define your autoencoder architecture here

  #Encoder first, can be a classical CNN with compresion of the spatial dimension

  #[...]

  #The latent space here must be 2D, corresponding to the following layer
  cnn.dense(nb_neurons=2, stric_size=1, activation=a_lin)

  #Then, the decoder, that should expand back the spatial dimension to the original image size

  #[...]

  #When connecting a dense layer to a conv layer you need to specify the input shape.
  #The number of neurons in your previous layer must correspond. (use strict_size=1 on a dense layer that connect to a conv layer)
  cnn.conv(f_size=[...], nb_filters=[...], padding=[...], activation=[...], input_shape=i_ar([DIM_W,DIM_H,DIM_D,NB_CHANNELS]))

  #To invert the effect of a pooling layer or a stride=2 convolutional layer you can add internal padding to a conv layer.
  #Here the following setup expand by a factor of 2 in each spatial dimensions

  #The decoder last layer must have the same number of channels that the input (here 1)
  # and an activation function that covers the possible input range (here normalized to LIN or LOGI are fine)
  cnn.conv(f_size=[...], nb_filters=1 , padding=[...], activation="LOGI")


for i in range(load_step,100):

  t = Thread(target=data_augm_fct)
  t.start()

  cnn.train(nb_iter=1, learning_rate=VAL, end_learning_rate=VAL, lr_decay=VAL, momentum=VAL, confmat=0, save_every=50, TC_scale_factor=16.0)

  t.join()

  if(i == 0):
    cnn.perf_eval()

  cnn.swap_data_buffers("TRAIN")

  #Draw the input/targets and predictions for a subset of the valid/test sample every 10 iterations
  if((i+1)%10 == 0):
    patch_in = np.zeros((image_size,image_size))
    patch_out = np.zeros((image_size,image_size))

    cnn.forward(saving=2, no_error=1, silent=1, drop_mode="AVG_MODEL")
    fwd_dat = np.fromfile("fwd_res/net0_%04d.dat"%(i+1), dtype="float32")
    fwd_dat = np.reshape(fwd_dat,(32,image_size*image_size))

    fig, axs = plt.subplots(5, 4, figsize=(6,7.5), dpi=int(image_size*2.0), constrained_layout=True)
    for j in range(0,2*5):
      axs[int(j/2)][int((j%2)*2)].axis('off')
      axs[int(j/2)][int((j%2)*2+1)].axis('off')

      fig.suptitle("Epoch %d"%(i+1))

    for j in range(0,2*5):
      patch_in[:,:] = np.reshape(fwd_dat[j][:],(image_size,image_size))
      patch_out[:,:] = np.reshape(data_valid[j][:],(image_size,image_size))

      axs[int(j/2)][int((j%2)*2)].imshow(patch_in, interpolation="bilinear", cmap="Greys")
      axs[int(j/2)][int((j%2)*2+1)].imshow(patch_out, interpolation="bilinear", cmap="Greys")

    plt.savefig("fig/fwd_%04d.png"%(i+1), dpi=int(image_size*2.0))
    plt.close()

EOF

### Visualize the latent space

In [ ]:
%%shell

cd /content/

python3 - <<EOF


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cmap
from matplotlib.lines import Line2D
from threading import Thread
import os, sys, glob

import albumentations as A
import cv2

sys.path.insert(0,glob.glob('/content/CIANNA/src/build/lib.*/')[-1])
import CIANNA as cnn

def i_ar(int_list):
	return np.array(int_list, dtype="int")

def f_ar(float_list):
	return np.array(float_list, dtype="float32")


if(not os.path.isdir("mnist_dat")):
	os.system("wget https://share.obspm.fr/s/EkYR5B2Wc2gNis3/download/mnist.tar.gz")
	os.system("tar -xvzf mnist.tar.gz")

image_size = 28

data = np.fromfile("mnist_dat/mnist_input.dat", dtype="float32")
data = np.reshape(data, (80000,image_size*image_size))*0.95+0.025
target = np.fromfile("mnist_dat/mnist_target.dat", dtype="float32")
target = np.reshape(target, (80000,10))

data_valid = data[70000:,:]

cnn.init(in_dim=i_ar([image_size,image_size]), in_nb_ch=1, out_dim=2,
		bias=0.1, b_size=32, comp_meth="C_CUDA", #Change to C_BLAS or C_NAIV
		dynamic_load=1, mixed_precision="FP16C_FP32A")

cnn.create_dataset("TEST", size=10000, input=data_valid, target=np.zeros((10000,2), dtype="float32"))

load_step = 0
if(load_step > 0):
  #Specify the number of layers to load from the autoencoder save file to stop at the latent space
	cnn.load("net_save/net0_s%04d.dat"%(load_step), nb_layers=8, iteration=load_step)

cnn.forward(saving=2, no_error=1, silent=0, drop_mode="AVG_MODEL")
fwd_dat = np.fromfile("fwd_res/net0_%04d.dat"%(load_step), dtype="float32")
fwd_dat = np.reshape(fwd_dat,(10000,3))

plt.scatter(fwd_dat[:,0], fwd_dat[:,1], s=0.4,
	c=cmap.tab10(np.argmax(target[70000:,:], axis=1)))

handles, labels = plt.gca().get_legend_handles_labels()

for i in range(0,10):
	handles.append(Line2D([0], [0], marker='o', color=cmap.tab10(i), markersize=8))
	labels.append("%d"%(i))

plt.legend(handles, labels)
plt.savefig("latent_space_visual.png")

EOF

In [ ]:
%cd /content/

from PIL import Image
import matplotlib.pyplot as plt

im = Image.open("latent_space_visual.png")
plt.figure(dpi=200)
plt.imshow(im)
plt.gca().axis('off')
plt.show()

### Generate new digits using the decoder from the latent space

In order to run only the decoder part, it is necessary to generate a save_file model that only contains the decoder layers. This can be done by opening the full autoencoder save file (net_save/net0_s%04d.dat) in ascii and deleting all the layers corresponding to the encoder. The input shape also need to be updated to correspond to the size of the latent space.

In [ ]:
%%shell

cd /content/

python3 - <<EOF


import numpy as np
import matplotlib.pyplot as plt
from threading import Thread
import os, sys, glob

import albumentations as A
import cv2

sys.path.insert(0,glob.glob('/content/CIANNA/src/build/lib.*/')[-1])
import CIANNA as cnn

if(not os.path.isdir("mnist_dat")):
	os.system("wget https://share.obspm.fr/s/EkYR5B2Wc2gNis3/download/mnist.tar.gz")
	os.system("tar -xvzf mnist.tar.gz")

def i_ar(int_list):
	return np.array(int_list, dtype="int")

def f_ar(float_list):
	return np.array(float_list, dtype="float32")

nb_r = 8
nb_theta = 60

r_range = [3,25]
theta_range = [0,2*np.pi]

radius_sampling = np.linspace(r_range[0],r_range[1],nb_r)
theta_sampling = np.linspace(theta_range[0],theta_range[1],nb_theta,endpoint=False)

lat_coords = np.zeros((nb_r,nb_theta,2))

for i in range(0,nb_r):
	r = radius_sampling[i]
	for j in range(0,nb_theta):
		theta = theta_sampling[j]
		lat_coords[i,j,:] = [r*np.cos(theta), r*np.sin(theta)]

plt.scatter(lat_coords[:,:,0].flatten(), lat_coords[:,:,1].flatten())
plt.show()

image_size = 28

empty_targets = np.zeros((nb_r*nb_theta,image_size*image_size), dtype="float32")

cnn.init(in_dim=i_ar([2]), in_nb_ch=1, out_dim=image_size*image_size,
		bias=0.1, b_size=32, comp_meth="C_CUDA", #Change to C_BLAS or C_NAIV
		dynamic_load=1, mixed_precision="FP16C_FP32A")

cnn.create_dataset("TEST", size=nb_r*nb_theta, input=f_ar(np.reshape(lat_coords,(nb_r*nb_theta,2))), target=empty_targets)

cnn.load("MNIST_decoder.dat", iteration=0)

cnn.forward(saving=2, no_error=1, silent=0, drop_mode="AVG_MODEL")
fwd_dat = np.fromfile("fwd_res/net0_%04d.dat"%(0), dtype="float32")
fwd_dat = np.reshape(fwd_dat,(nb_r*nb_theta,image_size*image_size))

fig, axs = plt.subplots(nb_theta, nb_r, figsize=(nb_r*1, nb_theta*1), layout="constrained")

for ax in axs.flatten():
	ax.axis("off")

for j in range(0,nb_theta):
	for i in range(0,nb_r):
		axs[j,i].imshow(fwd_dat[i*nb_theta+j].reshape((image_size,image_size)), cmap="Greys")

plt.savefig("generated_digits.png",dpi=100)

EOF

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

im = Image.open("generated_digits.png")
plt.figure(dpi=800)
plt.imshow(im)
plt.gca().axis('off')
plt.show()